## A comparison between Haar Cascade Classifier and Multi-task Cascaded Convolutional Neural Networks (MTCNN) for face detection task

### Haar cascade classifier for face detection
At the first step we try the CascadeClassifier class from OpenCV module to detect faces and estimate its precision
and recall. 

The algorithm is based on AdaBoost classifier trained on Haar features of the faces. The complete description of the method may be found on the OpenCV website: https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_objdetect/py_face_detection/py_face_detection.html

In [1]:
# Import modules
from cv2 import imread, imshow, CascadeClassifier
from scipy.io import loadmat
from iou import get_iou # Function placed in a separate iou.py file for convenience
import numpy as np
import pandas as pd
import os

In [2]:
# Define paths to WIDERFACE data
ground_truth_dir = './wider_face_data/wider_face_data/wider_face_split/'
ground_truth_images_dir = './wider_face_data/wider_face_data/WIDER_val/images/'

There are pre-trained Haar cascade classifier models for face detection in open-source repositories, e.g.: 
https://github.com/opencv/opencv/tree/master/data/haarcascades

The weights used in our example can be downloaded at this link: 
https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml

In [3]:
# Load the pre-trained CascadeClassifier model
classifier_haar = CascadeClassifier('haarcascade_frontalface_default.xml')

In this example, we validate the model on all the images together. However, the performance of the model may depend on the situations these images are subgrouped. The sizes of faces, the position of faces, the color parameters of the images all determine model performance. 

Ideas for future work: 

1) Train the model on the subgroups of people

2) Estimate the model performance on different face positions and different color characteristics of the images


In [4]:
# Load filenames and ground truth from validation set (.mat file)
# Initialize path to groundtruth
face_val_dict = loadmat(ground_truth_dir + 'wider_face_val.mat')

# Create the list of filenames
face_val_filenames = face_val_dict['file_list']
face_val_filenames_list = []

for i in range(np.shape(face_val_filenames)[0]):
    for k in range(np.shape(face_val_filenames[i, 0])[0]):
        face_val_filenames_list.append(face_val_filenames[i, 0][k, 0][0])

print('The length of validation file name list is: ', len(face_val_filenames_list))

# Create the list of coordinates of bounding boxes (bbx)
face_val_bbx = face_val_dict['face_bbx_list']
face_val_bbx_list = []

for i in range(np.shape(face_val_bbx)[0]):
    for k in range(np.shape(face_val_bbx[i, 0])[0]):
        face_val_bbx_list.append(face_val_bbx[i, 0][k, 0])

print('The length of validation bbx list is: ', len(face_val_bbx_list))

The length of validation file name list is:  3226
The length of validation bbx list is:  3226


The model performance metrics used in this example are detection threshold-constrained precision and recall. This means that the face is considered correctly identified only if it meets the pre-set threshold of the Intersection over Union (IoU) with the ground truth faces, and then precision and recall are computed. 

The IoU Python function is placed in a separate iou.py file for convenience.

In [5]:
# Go through the images in the validation set and compute validation error
# Define as a function
def evaluate_the_model(model,
                       model_type,
                       detection_threshold, 
                       images_path = ground_truth_images_dir):
    
    # Initialize variables for model performance estimation
    # In this example we use the estimates for all the images together
    total_groundtruth_faces = 0
    total_detected_faces = 0
    total_true_faces_count = 0

    # Count errors
    no_face_images_count = 0
    other_error_count = 0

    # Go through the images
    for image_index in range(len(face_val_filenames_list)):

        try:
            print('Analysing image ', image_index + 1, 'out of ', len(face_val_filenames_list) + 1)

            # Load the image
            file_name = face_val_filenames_list[image_index]
            image_folder_list = os.listdir(images_path)
            subfolder_name = '--'.join([i for i in file_name.split('_')[:2]])
            subfolder_name = [folder_name for folder_name in image_folder_list if folder_name.startswith(subfolder_name)][0]
            image_filepath = images_path + subfolder_name + '/' + file_name + '.jpg'
            image = imread(image_filepath)

            # Get groundtruth bbx
            bbx_groundtruth = face_val_bbx_list[image_index]
            bbx_groundtruth = [[bbx[0], bbx[0] + bbx[2], bbx[1], bbx[1] + bbx[3]] for bbx in bbx_groundtruth]

            # If image has no groundtruth faces, skip it and add 1 to no_face_images_count
            if bbx_groundtruth == [[0, 0, 0, 0]]:
                no_face_images_count += 1
                print('Image with no faces detected, skip and continue...')
                continue

            # Find bbx with the model
            if model_type == 'haar':
                bbx_model = model.detectMultiScale(image)
            
            elif model_type == 'mtcnn':
                bbx_model = model.detect_faces(image)
                bbx_model = [el['box'] for el in bbx_model] 
                
            else: 
                print('Model is not in the list')

            bbx_model = [[bbx[0], bbx[0] + bbx[2], bbx[1], bbx[1] + bbx[3]] for bbx in bbx_model]

            # Check which detected bbx overlap with ground truth not more than the threshold
            # Go throght detected faces
            true_faces = []
            for i in range(np.shape(bbx_model)[0]):
                bbx1 = {
                       'x1': bbx_model[i][0],
                       'x2': bbx_model[i][1],
                       'y1': bbx_model[i][2],
                       'y2': bbx_model[i][3]}

                # For each detected face, go through the true faces
                # And find one that is most similar
                bbx_iou_k = []

                for k in range(np.shape(bbx_groundtruth)[0]):
                    bbx2 = {
                           'x1': bbx_groundtruth[k][0],
                           'x2': bbx_groundtruth[k][1],
                           'y1': bbx_groundtruth[k][2],
                           'y2': bbx_groundtruth[k][3]}

                    bbx_iou = get_iou(bbx1, bbx2)
                    bbx_iou_k.append(bbx_iou)

                # This finds the most similar image to the ground truth (biggest IoU)
                bbx_iou_max = max(bbx_iou_k)

                # If the predicted face overlaps better than the 
                # threshold, append to true_faces
                if bbx_iou_max > detection_threshold:
                    true_faces.append(bbx_model[i])

            # Add to the count of classification
            total_groundtruth_faces += len(bbx_groundtruth)
            total_detected_faces += len(bbx_model)
            total_true_faces_count += len(true_faces)

        except:
            print('Other error detected, skip and continue...')
            other_error_count += 1

    total_false_faces_count = total_detected_faces - total_true_faces_count

    # Create a dictionary with output
    validation_errors = {
                        'no_face_errors': no_face_images_count, 
                        'other_errors': other_error_count}
    validation_counts = {
                        'total_groundtruth_faces': total_groundtruth_faces, 
                        'total_detected_faces': total_detected_faces, 
                        'total_true_faces': total_true_faces_count, 
                        'total_false_faces': total_false_faces_count}
    validation_report = {
                        'validation_errors': validation_errors, 
                        'validation_counts': validation_counts}

    return(validation_report)

In [6]:
# Run validation for haar Cascade Classifier, get the report
haar_validation_report = evaluate_the_model(model = classifier_haar, model_type = 'haar', detection_threshold=0.5)

Analysing image  1 out of  3227
Analysing image  2 out of  3227
Analysing image  3 out of  3227
Analysing image  4 out of  3227
Analysing image  5 out of  3227
Analysing image  6 out of  3227
Analysing image  7 out of  3227
Analysing image  8 out of  3227
Analysing image  9 out of  3227
Analysing image  10 out of  3227
Analysing image  11 out of  3227
Analysing image  12 out of  3227
Analysing image  13 out of  3227
Analysing image  14 out of  3227
Analysing image  15 out of  3227
Analysing image  16 out of  3227
Analysing image  17 out of  3227
Analysing image  18 out of  3227
Analysing image  19 out of  3227
Analysing image  20 out of  3227
Analysing image  21 out of  3227
Analysing image  22 out of  3227
Analysing image  23 out of  3227
Analysing image  24 out of  3227
Analysing image  25 out of  3227
Analysing image  26 out of  3227
Analysing image  27 out of  3227
Analysing image  28 out of  3227
Analysing image  29 out of  3227
Analysing image  30 out of  3227
Image with no faces

Analysing image  243 out of  3227
Analysing image  244 out of  3227
Analysing image  245 out of  3227
Analysing image  246 out of  3227
Analysing image  247 out of  3227
Analysing image  248 out of  3227
Analysing image  249 out of  3227
Analysing image  250 out of  3227
Analysing image  251 out of  3227
Analysing image  252 out of  3227
Analysing image  253 out of  3227
Analysing image  254 out of  3227
Analysing image  255 out of  3227
Analysing image  256 out of  3227
Analysing image  257 out of  3227
Analysing image  258 out of  3227
Analysing image  259 out of  3227
Analysing image  260 out of  3227
Analysing image  261 out of  3227
Analysing image  262 out of  3227
Analysing image  263 out of  3227
Analysing image  264 out of  3227
Analysing image  265 out of  3227
Analysing image  266 out of  3227
Analysing image  267 out of  3227
Analysing image  268 out of  3227
Analysing image  269 out of  3227
Analysing image  270 out of  3227
Analysing image  271 out of  3227
Analysing imag

Analysing image  485 out of  3227
Analysing image  486 out of  3227
Analysing image  487 out of  3227
Analysing image  488 out of  3227
Analysing image  489 out of  3227
Analysing image  490 out of  3227
Analysing image  491 out of  3227
Analysing image  492 out of  3227
Analysing image  493 out of  3227
Analysing image  494 out of  3227
Analysing image  495 out of  3227
Analysing image  496 out of  3227
Analysing image  497 out of  3227
Analysing image  498 out of  3227
Analysing image  499 out of  3227
Analysing image  500 out of  3227
Analysing image  501 out of  3227
Analysing image  502 out of  3227
Analysing image  503 out of  3227
Analysing image  504 out of  3227
Analysing image  505 out of  3227
Analysing image  506 out of  3227
Analysing image  507 out of  3227
Analysing image  508 out of  3227
Analysing image  509 out of  3227
Analysing image  510 out of  3227
Analysing image  511 out of  3227
Analysing image  512 out of  3227
Analysing image  513 out of  3227
Analysing imag

Analysing image  727 out of  3227
Analysing image  728 out of  3227
Analysing image  729 out of  3227
Analysing image  730 out of  3227
Analysing image  731 out of  3227
Analysing image  732 out of  3227
Analysing image  733 out of  3227
Analysing image  734 out of  3227
Analysing image  735 out of  3227
Analysing image  736 out of  3227
Analysing image  737 out of  3227
Analysing image  738 out of  3227
Analysing image  739 out of  3227
Analysing image  740 out of  3227
Analysing image  741 out of  3227
Analysing image  742 out of  3227
Analysing image  743 out of  3227
Analysing image  744 out of  3227
Analysing image  745 out of  3227
Analysing image  746 out of  3227
Analysing image  747 out of  3227
Analysing image  748 out of  3227
Analysing image  749 out of  3227
Analysing image  750 out of  3227
Analysing image  751 out of  3227
Analysing image  752 out of  3227
Analysing image  753 out of  3227
Analysing image  754 out of  3227
Analysing image  755 out of  3227
Analysing imag

Analysing image  967 out of  3227
Analysing image  968 out of  3227
Analysing image  969 out of  3227
Analysing image  970 out of  3227
Analysing image  971 out of  3227
Analysing image  972 out of  3227
Analysing image  973 out of  3227
Analysing image  974 out of  3227
Analysing image  975 out of  3227
Analysing image  976 out of  3227
Analysing image  977 out of  3227
Analysing image  978 out of  3227
Analysing image  979 out of  3227
Analysing image  980 out of  3227
Analysing image  981 out of  3227
Analysing image  982 out of  3227
Analysing image  983 out of  3227
Analysing image  984 out of  3227
Analysing image  985 out of  3227
Analysing image  986 out of  3227
Analysing image  987 out of  3227
Analysing image  988 out of  3227
Analysing image  989 out of  3227
Analysing image  990 out of  3227
Analysing image  991 out of  3227
Analysing image  992 out of  3227
Analysing image  993 out of  3227
Analysing image  994 out of  3227
Analysing image  995 out of  3227
Analysing imag

...

Analysing image  2142 out of  3227
Analysing image  2143 out of  3227
Analysing image  2144 out of  3227
Analysing image  2145 out of  3227
Analysing image  2146 out of  3227
Analysing image  2147 out of  3227
Analysing image  2148 out of  3227
Analysing image  2149 out of  3227
Analysing image  2150 out of  3227
Analysing image  2151 out of  3227
Analysing image  2152 out of  3227
Analysing image  2153 out of  3227
Analysing image  2154 out of  3227
Analysing image  2155 out of  3227
Analysing image  2156 out of  3227
Analysing image  2157 out of  3227
Analysing image  2158 out of  3227
Analysing image  2159 out of  3227
Analysing image  2160 out of  3227
Analysing image  2161 out of  3227
Analysing image  2162 out of  3227
Analysing image  2163 out of  3227
Analysing image  2164 out of  3227
Analysing image  2165 out of  3227
Analysing image  2166 out of  3227
Analysing image  2167 out of  3227
Analysing image  2168 out of  3227
Analysing image  2169 out of  3227
Analysing image  217

Analysing image  2377 out of  3227
Analysing image  2378 out of  3227
Analysing image  2379 out of  3227
Analysing image  2380 out of  3227
Analysing image  2381 out of  3227
Analysing image  2382 out of  3227
Analysing image  2383 out of  3227
Analysing image  2384 out of  3227
Analysing image  2385 out of  3227
Analysing image  2386 out of  3227
Analysing image  2387 out of  3227
Analysing image  2388 out of  3227
Analysing image  2389 out of  3227
Analysing image  2390 out of  3227
Analysing image  2391 out of  3227
Analysing image  2392 out of  3227
Analysing image  2393 out of  3227
Analysing image  2394 out of  3227
Analysing image  2395 out of  3227
Analysing image  2396 out of  3227
Analysing image  2397 out of  3227
Analysing image  2398 out of  3227
Analysing image  2399 out of  3227
Analysing image  2400 out of  3227
Analysing image  2401 out of  3227
Analysing image  2402 out of  3227
Analysing image  2403 out of  3227
Analysing image  2404 out of  3227
Analysing image  240

Analysing image  2611 out of  3227
Analysing image  2612 out of  3227
Analysing image  2613 out of  3227
Analysing image  2614 out of  3227
Analysing image  2615 out of  3227
Analysing image  2616 out of  3227
Analysing image  2617 out of  3227
Analysing image  2618 out of  3227
Analysing image  2619 out of  3227
Analysing image  2620 out of  3227
Analysing image  2621 out of  3227
Analysing image  2622 out of  3227
Analysing image  2623 out of  3227
Analysing image  2624 out of  3227
Analysing image  2625 out of  3227
Analysing image  2626 out of  3227
Analysing image  2627 out of  3227
Analysing image  2628 out of  3227
Analysing image  2629 out of  3227
Analysing image  2630 out of  3227
Analysing image  2631 out of  3227
Analysing image  2632 out of  3227
Analysing image  2633 out of  3227
Analysing image  2634 out of  3227
Analysing image  2635 out of  3227
Analysing image  2636 out of  3227
Analysing image  2637 out of  3227
Analysing image  2638 out of  3227
Analysing image  263

Analysing image  2846 out of  3227
Analysing image  2847 out of  3227
Analysing image  2848 out of  3227
Analysing image  2849 out of  3227
Analysing image  2850 out of  3227
Analysing image  2851 out of  3227
Analysing image  2852 out of  3227
Analysing image  2853 out of  3227
Analysing image  2854 out of  3227
Analysing image  2855 out of  3227
Analysing image  2856 out of  3227
Analysing image  2857 out of  3227
Analysing image  2858 out of  3227
Analysing image  2859 out of  3227
Analysing image  2860 out of  3227
Analysing image  2861 out of  3227
Analysing image  2862 out of  3227
Analysing image  2863 out of  3227
Analysing image  2864 out of  3227
Analysing image  2865 out of  3227
Analysing image  2866 out of  3227
Analysing image  2867 out of  3227
Analysing image  2868 out of  3227
Analysing image  2869 out of  3227
Analysing image  2870 out of  3227
Analysing image  2871 out of  3227
Analysing image  2872 out of  3227
Analysing image  2873 out of  3227
Analysing image  287

Analysing image  3041 out of  3227
Analysing image  3042 out of  3227
Analysing image  3043 out of  3227
Analysing image  3044 out of  3227
Analysing image  3045 out of  3227
Analysing image  3046 out of  3227
Analysing image  3047 out of  3227
Analysing image  3048 out of  3227
Analysing image  3049 out of  3227
Analysing image  3050 out of  3227
Analysing image  3051 out of  3227
Analysing image  3052 out of  3227
Analysing image  3053 out of  3227
Analysing image  3054 out of  3227
Analysing image  3055 out of  3227
Analysing image  3056 out of  3227
Analysing image  3057 out of  3227
Analysing image  3058 out of  3227
Analysing image  3059 out of  3227
Analysing image  3060 out of  3227
Analysing image  3061 out of  3227
Analysing image  3062 out of  3227
Analysing image  3063 out of  3227
Analysing image  3064 out of  3227
Analysing image  3065 out of  3227
Analysing image  3066 out of  3227
Analysing image  3067 out of  3227
Analysing image  3068 out of  3227
Analysing image  306

Print the error summary. In the current project the errors are not analyzed. In the future work all the errors should be analyzed instead of using 'try-catch' block. 

In [7]:
# Print how many errors found 
print('Found images with no true faces: ', haar_validation_report['validation_errors']['no_face_errors'], ' out of ', len(face_val_filenames_list))
print('Found other errors: ', haar_validation_report['validation_errors']['other_errors'], ' out of ', len(face_val_filenames_list))

Found images with no true faces:  4  out of  3226
Found other errors:  37  out of  3226


Compute precision and recall of the model

$precision = \frac{TP}{TP + FP}$

$recall = \frac{TP}{TP + FN}$

TP are the faces correctly predicted as faces

FP are the predicted faces that are not faces (something else)

FN in our case is the difference between all faces in the images and the faces correctly detected (TP)


Ideas for future work: 

1) By changing Intersection-over-Union (IoU) detection threshold and plotting it against the precision and recall we can find the threshold for optimal precision and recall

2) Experiment with another metric. For example, Localization Recall Precision (LRP) [Oksuz 2018]: https://arxiv.org/abs/1807.01696

In [8]:
# Compute recision and recall of the model
model_precision = haar_validation_report['validation_counts']['total_true_faces'] / (haar_validation_report['validation_counts']['total_true_faces'] + haar_validation_report['validation_counts']['total_false_faces'])

model_recall = haar_validation_report['validation_counts']['total_true_faces'] / haar_validation_report['validation_counts']['total_groundtruth_faces']

print('Haar cascade model precision is: ', round(model_precision, 2))
print('Haar cascade model recall (sensitivity) is: ', round(model_recall, 2))

Haar cascade model precision is:  0.54
Haar cascade model recall (sensitivity) is:  0.17


### MTCNN for face detection
At the second step we run Multitask Cascaded Convolutional Networks (MTCNN) to detect faces. 
An MTCNN is an architecture that combines three neural networks to suggest the best bbx for faces. 
Each of the three models are trained separately. For this reason, it is very difficult to train from scratch, but pre-trained models are available. 
For this project we use the following implementation of MTCNN: https://github.com/jbrownlee/mtcnn . 
MTCNN was developed in 2016 [https://arxiv.org/abs/1604.02878] and today is considered one of the best-performing models. 


In [9]:
# Load the module mtcnn and initialize classifier
import mtcnn
classifier_mtcnn = mtcnn.MTCNN() # The library uses pre-trained model by default

Using TensorFlow backend.
/home/saltair/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/saltair/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/saltair/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/saltair/anaconda3/lib/python3.7/site-packages/t

/home/saltair/anaconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/saltair/anaconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/saltair/anaconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/saltair/anaconda3/lib/python3.7/site-packages/tensor

In [10]:
# Run validation for MTCNN, get the report
# Use the method of validation previously defined
mtcnn_validation_report = evaluate_the_model(model = classifier_mtcnn, model_type = 'mtcnn', detection_threshold=0.5)

Analysing image  1 out of  3227

Analysing image  2 out of  3227
Analysing image  3 out of  3227
Analysing image  4 out of  3227
Analysing image  5 out of  3227
Analysing image  6 out of  3227
Analysing image  7 out of  3227
Analysing image  8 out of  3227
Analysing image  9 out of  3227
Analysing image  10 out of  3227
Analysing image  11 out of  3227
Analysing image  12 out of  3227
Analysing image  13 out of  3227
Analysing image  14 out of  3227
Analysing image  15 out of  3227
Analysing image  16 out of  3227
Analysing image  17 out of  3227
Analysing image  18 out of  3227
Analysing image  19 out of  3227
Analysing image  20 out of  3227
Analysing image  21 out of  3227
Analysing image  22 out of  3227
Analysing image  23 out of  3227
Analysing image  24 out of  3227
Analysing image  25 out of  3227
Analysing image  26 out of  3227
Analysing image  27 out of  3227
Analysing image  28 out of  3227
Analysing image  29 out of  3227
Analysing image  30 out of  3227
Image with no face

Analysing image  239 out of  3227
Analysing image  240 out of  3227
Analysing image  241 out of  3227
Analysing image  242 out of  3227
Analysing image  243 out of  3227
Analysing image  244 out of  3227
Analysing image  245 out of  3227
Analysing image  246 out of  3227
Analysing image  247 out of  3227
Analysing image  248 out of  3227
Analysing image  249 out of  3227
Analysing image  250 out of  3227
Analysing image  251 out of  3227
Analysing image  252 out of  3227
Analysing image  253 out of  3227
Analysing image  254 out of  3227
Analysing image  255 out of  3227
Analysing image  256 out of  3227
Analysing image  257 out of  3227
Analysing image  258 out of  3227
Analysing image  259 out of  3227
Analysing image  260 out of  3227
Analysing image  261 out of  3227
Analysing image  262 out of  3227
Analysing image  263 out of  3227
Analysing image  264 out of  3227
Analysing image  265 out of  3227
Analysing image  266 out of  3227
Analysing image  267 out of  3227
Analysing imag

Analysing image  480 out of  3227
Analysing image  481 out of  3227
Analysing image  482 out of  3227
Analysing image  483 out of  3227
Analysing image  484 out of  3227
Analysing image  485 out of  3227
Analysing image  486 out of  3227
Analysing image  487 out of  3227
Analysing image  488 out of  3227
Analysing image  489 out of  3227
Analysing image  490 out of  3227
Analysing image  491 out of  3227
Analysing image  492 out of  3227
Analysing image  493 out of  3227
Analysing image  494 out of  3227
Analysing image  495 out of  3227
Analysing image  496 out of  3227
Analysing image  497 out of  3227
Analysing image  498 out of  3227
Analysing image  499 out of  3227
Analysing image  500 out of  3227
Analysing image  501 out of  3227
Analysing image  502 out of  3227
Analysing image  503 out of  3227
Analysing image  504 out of  3227
Analysing image  505 out of  3227
Analysing image  506 out of  3227
Analysing image  507 out of  3227
Analysing image  508 out of  3227
Analysing imag

Analysing image  721 out of  3227
Analysing image  722 out of  3227
Analysing image  723 out of  3227
Analysing image  724 out of  3227
Analysing image  725 out of  3227
Analysing image  726 out of  3227
Analysing image  727 out of  3227
Analysing image  728 out of  3227
Analysing image  729 out of  3227
Analysing image  730 out of  3227
Analysing image  731 out of  3227
Analysing image  732 out of  3227
Analysing image  733 out of  3227
Analysing image  734 out of  3227
Analysing image  735 out of  3227
Analysing image  736 out of  3227
Analysing image  737 out of  3227
Analysing image  738 out of  3227
Analysing image  739 out of  3227
Analysing image  740 out of  3227
Analysing image  741 out of  3227
Analysing image  742 out of  3227
Analysing image  743 out of  3227
Analysing image  744 out of  3227
Analysing image  745 out of  3227
Analysing image  746 out of  3227
Analysing image  747 out of  3227
Analysing image  748 out of  3227
Analysing image  749 out of  3227
Analysing imag

Analysing image  960 out of  3227
Analysing image  961 out of  3227
Analysing image  962 out of  3227
Analysing image  963 out of  3227
Analysing image  964 out of  3227
Analysing image  965 out of  3227
Analysing image  966 out of  3227
Analysing image  967 out of  3227
Analysing image  968 out of  3227
Analysing image  969 out of  3227
Analysing image  970 out of  3227
Analysing image  971 out of  3227
Analysing image  972 out of  3227
Analysing image  973 out of  3227
Analysing image  974 out of  3227
Analysing image  975 out of  3227
Analysing image  976 out of  3227
Analysing image  977 out of  3227
Analysing image  978 out of  3227
Analysing image  979 out of  3227
Analysing image  980 out of  3227
Analysing image  981 out of  3227
Analysing image  982 out of  3227
Analysing image  983 out of  3227
Analysing image  984 out of  3227
Analysing image  985 out of  3227
Analysing image  986 out of  3227
Analysing image  987 out of  3227
Analysing image  988 out of  3227
Analysing imag

...

Analysing image  2134 out of  3227
Analysing image  2135 out of  3227
Analysing image  2136 out of  3227
Analysing image  2137 out of  3227
Analysing image  2138 out of  3227
Analysing image  2139 out of  3227
Analysing image  2140 out of  3227
Analysing image  2141 out of  3227
Analysing image  2142 out of  3227
Analysing image  2143 out of  3227
Analysing image  2144 out of  3227
Analysing image  2145 out of  3227
Analysing image  2146 out of  3227
Analysing image  2147 out of  3227
Analysing image  2148 out of  3227
Analysing image  2149 out of  3227
Analysing image  2150 out of  3227
Analysing image  2151 out of  3227
Analysing image  2152 out of  3227
Analysing image  2153 out of  3227
Analysing image  2154 out of  3227
Analysing image  2155 out of  3227
Analysing image  2156 out of  3227
Analysing image  2157 out of  3227
Analysing image  2158 out of  3227
Analysing image  2159 out of  3227
Analysing image  2160 out of  3227
Analysing image  2161 out of  3227
Analysing image  216

Analysing image  2369 out of  3227
Analysing image  2370 out of  3227
Analysing image  2371 out of  3227
Analysing image  2372 out of  3227
Analysing image  2373 out of  3227
Analysing image  2374 out of  3227
Analysing image  2375 out of  3227
Analysing image  2376 out of  3227
Analysing image  2377 out of  3227
Analysing image  2378 out of  3227
Analysing image  2379 out of  3227
Analysing image  2380 out of  3227
Analysing image  2381 out of  3227
Analysing image  2382 out of  3227
Analysing image  2383 out of  3227
Analysing image  2384 out of  3227
Analysing image  2385 out of  3227
Analysing image  2386 out of  3227
Analysing image  2387 out of  3227
Analysing image  2388 out of  3227
Analysing image  2389 out of  3227
Analysing image  2390 out of  3227
Analysing image  2391 out of  3227
Analysing image  2392 out of  3227
Analysing image  2393 out of  3227
Analysing image  2394 out of  3227
Analysing image  2395 out of  3227
Analysing image  2396 out of  3227
Analysing image  239

Analysing image  2602 out of  3227
Analysing image  2603 out of  3227
Analysing image  2604 out of  3227
Analysing image  2605 out of  3227
Analysing image  2606 out of  3227
Analysing image  2607 out of  3227
Analysing image  2608 out of  3227
Analysing image  2609 out of  3227
Analysing image  2610 out of  3227
Analysing image  2611 out of  3227
Analysing image  2612 out of  3227
Analysing image  2613 out of  3227
Analysing image  2614 out of  3227
Analysing image  2615 out of  3227
Analysing image  2616 out of  3227
Analysing image  2617 out of  3227
Analysing image  2618 out of  3227
Analysing image  2619 out of  3227
Analysing image  2620 out of  3227
Analysing image  2621 out of  3227
Analysing image  2622 out of  3227
Analysing image  2623 out of  3227
Analysing image  2624 out of  3227
Analysing image  2625 out of  3227
Analysing image  2626 out of  3227
Analysing image  2627 out of  3227
Analysing image  2628 out of  3227
Analysing image  2629 out of  3227
Analysing image  263

Analysing image  2837 out of  3227
Analysing image  2838 out of  3227
Analysing image  2839 out of  3227
Analysing image  2840 out of  3227
Analysing image  2841 out of  3227
Analysing image  2842 out of  3227
Analysing image  2843 out of  3227
Analysing image  2844 out of  3227
Analysing image  2845 out of  3227
Analysing image  2846 out of  3227
Analysing image  2847 out of  3227
Analysing image  2848 out of  3227
Analysing image  2849 out of  3227
Analysing image  2850 out of  3227
Analysing image  2851 out of  3227
Analysing image  2852 out of  3227
Analysing image  2853 out of  3227
Analysing image  2854 out of  3227
Analysing image  2855 out of  3227
Analysing image  2856 out of  3227
Analysing image  2857 out of  3227
Analysing image  2858 out of  3227
Analysing image  2859 out of  3227
Analysing image  2860 out of  3227
Analysing image  2861 out of  3227
Analysing image  2862 out of  3227
Analysing image  2863 out of  3227
Analysing image  2864 out of  3227
Analysing image  286

Analysing image  3033 out of  3227
Analysing image  3034 out of  3227
Analysing image  3035 out of  3227
Analysing image  3036 out of  3227
Analysing image  3037 out of  3227
Analysing image  3038 out of  3227
Analysing image  3039 out of  3227
Analysing image  3040 out of  3227
Analysing image  3041 out of  3227
Analysing image  3042 out of  3227
Analysing image  3043 out of  3227
Analysing image  3044 out of  3227
Analysing image  3045 out of  3227
Analysing image  3046 out of  3227
Analysing image  3047 out of  3227
Analysing image  3048 out of  3227
Analysing image  3049 out of  3227
Analysing image  3050 out of  3227
Analysing image  3051 out of  3227
Analysing image  3052 out of  3227
Analysing image  3053 out of  3227
Analysing image  3054 out of  3227
Analysing image  3055 out of  3227
Analysing image  3056 out of  3227
Analysing image  3057 out of  3227
Analysing image  3058 out of  3227
Analysing image  3059 out of  3227
Analysing image  3060 out of  3227
Analysing image  306

In [11]:
# Compute recision and recall of the model
mtcnn_model_precision = mtcnn_validation_report['validation_counts']['total_true_faces'] / (mtcnn_validation_report['validation_counts']['total_true_faces'] + mtcnn_validation_report['validation_counts']['total_false_faces'])

mtcnn_model_recall = mtcnn_validation_report['validation_counts']['total_true_faces'] / mtcnn_validation_report['validation_counts']['total_groundtruth_faces']

print('Haar cascade model precision is: ', round(mtcnn_model_precision, 2))
print('Haar cascade model recall (sensitivity) is: ', round(mtcnn_model_recall, 2))

Haar cascade model precision is:  0.92
Haar cascade model recall (sensitivity) is:  0.23


### Discussion

#### Comparing model performance
It is shown that MTCNN model has much better performance in terms od IoU-constrained precision and recall on a face detection task,  compared to Haar Cascade Classifyer with the same IoU detection threshold. The difference between the methods is the biggest in terms of precision: the recognised objects are much more likely to be faces when MTCNN model is used. This example demonstrates that the approach based on Convolutional Neural Networks is capable of detecting features in the images much better compared to other techniques. A bad recall in both models may be due to the fact that most faces in the images were difficult to detect for a variety of reasons. 

#### What if no ground truth is available? 
In the case the ground truth is not available, MTCNN has advantage over Haar Cascade Classifier, too. Apart from returning bbx, the model also returns the probabilities that the detected objects are faces. If higher precision is required, a threshold may be put on probabilities as well. In this case, however, the recall (sensitivity) of the model may be lower. In our example no threshold for probability is used.

To evaluate the model without ground truth, it may be helpful to use another model to return the probabilities of the objects being detected. To do this, the bbx generated by the [face detection] model may be used to crop the images ad then analyse the cropped objects with another model (for example, a VGG-type model trained on people with/without masks).

Alternatively, the ground truth may be generated using Amazon SageMaker Ground Truth: https://aws.amazon.com/sagemaker/groundtruth/ . The service uses examples of ground truth and both automatic and human labelers (depending on the choice) to label the ground truth on the images. 

#### Improving the results 
Pre-trained MTCNN showed top performance for face detection on a large dataset of images. If other objects need to be detected, it is better to use a different approach. For example, YOLO or SSD can be trained from scratch for the necessary objects. Often it is better to train an already pre-trained neural network on a custom dataset, or use some of the layers from a pre-trained neural network. 

Alternatively, Haar feature learning with AdaBoost may be trained from scratch for the necessary purpose.